In [11]:
def normalize_english(text):
    def fn(m):
        word = m.group()
        if word in english_dictionary:
            return english_dictionary.get(word)
        else:
            return word

    text = re.sub("([A-Za-z]+)", fn, text)
    return text

In [12]:
normalize_english("abc123A")

NameError: name 'english_dictionary' is not defined

In [9]:
def normalize_upper(text):
    text = text.group(0)

    if all([char.isupper() for char in text]):
        return "".join(upper_to_kor[char] for char in text)
    else:
        return text

In [10]:
normalize_upper("abc")

AttributeError: 'str' object has no attribute 'group'

In [8]:
import re
re.findall("([A-Za-z]+)","(ABc23일)")

['ABc']

클리너에서 텍스트를 처리
클리너에서의  코리안 클리너를 사용
코리안 클리너에서는 from tensorflow_tts.utils.korean import tokenize as ko_tokenize를 활용
https://github.com/TensorSpeech/TensorFlowTTS/blob/136877136355c82d7ba474ceb7a8f133bd84767e/tensorflow_tts/utils/korean.py#L349

In [ ]:
import soundfile as sf
import numpy as np

import tensorflow as tf

from tensorflow_tts.inference import AutoProcessor
from tensorflow_tts.inference import TFAutoModel

processor = AutoProcessor.from_pretrained("tensorspeech/tts-tacotron2-kss-ko")
input_ids = processor.text_to_sequence(text)
tacotron2 = TFAutoModel.from_pretrained("tensorspeech/tts-tacotron2-kss-ko")
mb_melgan = TFAutoModel.from_pretrained("tensorspeech/tts-mb_melgan-kss-ko")

In [99]:
text = "마징가Z 3화"

input_ids = processor.text_to_sequence(text)

# tacotron2 inference (text-to-mel)
decoder_output, mel_outputs, stop_token_prediction, alignment_history = tacotron2.inference(
    input_ids=tf.expand_dims(tf.convert_to_tensor(input_ids, dtype=tf.int32), 0),
    input_lengths=tf.convert_to_tensor([len(input_ids)], tf.int32),
    speaker_ids=tf.convert_to_tensor([0], dtype=tf.int32),
)

# melgan inference (mel-to-wav)
audio = mb_melgan.inference(mel_outputs)[0, :, 0]

# save to file
sf.write('./audio.wav', audio, 22050, "PCM_16")

import IPython.display as ipd
ipd.Audio('audio.wav') 

In [2]:
text = "마징가Z 13화 SS501 시즌5 one two 123 2005 123원 123호 12시"
from TN import tokenize
from unicode import join_jamos
merge_jamo = join_jamos(tokenize(text))
print(merge_jamo)

마징가제트 십삼화 더블에스오공일 시즌파이브 one two 백이십삼 이천오 백이십삼원 백이십삼호 열두시


In [4]:
def number_to_korean(num_str, is_count=False):
    if is_count:
        num_str, unit_str = num_str.group(1), num_str.group(2)
    else:
        num_str, unit_str = num_str.group(), ""

    num_str = num_str.replace(",", "")
    num = ast.literal_eval(num_str)

    if num == 0:
        return "영"

    check_float = num_str.split(".")
    if len(check_float) == 2:
        digit_str, float_str = check_float
    elif len(check_float) >= 3:
        raise Exception(" [!] Wrong number format")
    else:
        digit_str, float_str = check_float[0], None

    if is_count and float_str is not None:
        raise Exception(" [!] `is_count` and float number does not fit each other")

    digit = int(digit_str)

    if digit_str.startswith("-"):
        digit, digit_str = abs(digit), str(abs(digit))

    kor = ""
    size = len(str(digit))
    tmp = []

    for i, v in enumerate(digit_str, start=1):
        v = int(v)

        if v != 0:
            if is_count:
                tmp += count_to_kor1[v]
            else:
                tmp += num_to_kor1[v]

            tmp += num_to_kor3[(size - i) % 4]

        if (size - i) % 4 == 0 and len(tmp) != 0:
            kor += "".join(tmp)
            tmp = []
            kor += num_to_kor2[int((size - i) / 4)]

    if is_count:
        if kor.startswith("한") and len(kor) > 1:
            kor = kor[1:]

        if any(word in kor for word in count_tenth_dict):
            kor = re.sub(
                "|".join(count_tenth_dict.keys()),
                lambda x: count_tenth_dict[x.group()],
                kor,
            )

    if not is_count and kor.startswith("일") and len(kor) > 1:
        kor = kor[1:]

    if float_str is not None:
        kor += "쩜 "
        kor += re.sub("\d", lambda x: num_to_kor[x.group()], float_str)

    if num_str.startswith("+"):
        kor = "플러스 " + kor
    elif num_str.startswith("-"):
        kor = "마이너스 " + kor

    return kor + unit_str

In [5]:
number_to_korean("+3", is_count=False)

AttributeError: 'str' object has no attribute 'group'

In [9]:
text = "3화 시즌3 1박2일 12번 째 사제 2번째 남편 12시 12등"
from TN import tokenize
from unicode import join_jamos
merge_jamo = join_jamos(tokenize(text))
print(merge_jamo)

삼화 시즌쓰리 일박이일 십이번 째 사제 이번째 남편 열두시 십이등
